In [13]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib

face_model = cv2.dnn.readNetFromTorch("models/openface.nn4.small2.v1.t7")

data = []
labels = []
dataset_path = "known_faces"

for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue
    for image_name in os.listdir(person_folder):
        image_path = os.path.join(person_folder, image_name)
        img = cv2.imread(image_path)
        if img is None:
            continue
        blob = cv2.dnn.blobFromImage(img, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        face_model.setInput(blob)
        vec = face_model.forward()
        data.append(vec.flatten())
        labels.append(person_name)

le = LabelEncoder()
labels_num = le.fit_transform(labels)
clf = SVC(probability=True)
clf.fit(data, labels_num)
joblib.dump((clf, le), "classifier.pkl")
print("Model trained and saved.")

Model trained and saved.


In [7]:
import cv2
import numpy as np
import os
from keras.models import load_model
import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from joblib import dump, load

TEST_MODE = False

face_model = cv2.dnn.readNetFromTorch("models/openface.nn4.small2.v1.t7")
face_detector = cv2.dnn.readNetFromCaffe("models/deploy.prototxt", "models/res10_300x300_ssd_iter_140000_fp16.caffemodel")
age_net = cv2.dnn.readNetFromCaffe("models/age_deploy.prototxt", "models/age_net.caffemodel")
gender_net = cv2.dnn.readNetFromCaffe("models/gender_deploy.prototxt", "models/gender_net.caffemodel")
emotion_model = load_model("models/emotion_model.hdf5", compile=False)

age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']
emotion_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

dataset_path = "known_faces"
names = []
embeddings = []

for person in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person)
    for image_name in os.listdir(person_folder):
        image_path = os.path.join(person_folder, image_name)
        image = cv2.imread(image_path)
        if image is None:
            continue
        h, w = image.shape[:2]
        blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        face_model.setInput(blob)
        embedding = face_model.forward()
        names.append(person)
        embeddings.append(embedding.flatten())

le = LabelEncoder()
labels = le.fit_transform(names)

clf = SVC(kernel='linear', probability=True)
clf.fit(embeddings, labels)

file_name = "attendance.xlsx"
if not os.path.exists(file_name):
    df = pd.DataFrame(columns=["Name", "Date", "Time", "Age", "Gender", "Emotion", "Status"])
    df.to_excel(file_name, index=False)

def mark_attendance(name, age, gender, emotion):
    now = datetime.datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M:%S")
    df = pd.read_excel(file_name)
    if not ((df["Name"] == name) & (df["Date"] == date_str)).any():
        new_entry = {"Name": name, "Date": date_str, "Time": time_str, "Age": age, "Gender": gender, "Emotion": emotion, "Status": "Present"}
        df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
        df.to_excel(file_name, index=False)

def mark_absent_for_missing_students():
    df = pd.read_excel(file_name)
    date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    present_students = df[df["Date"] == date_str]["Name"].unique()
    all_students = os.listdir(dataset_path)
    for student in all_students:
        if student not in present_students:
            now = datetime.datetime.now()
            time_str = now.strftime("%H:%M:%S")
            absent_entry = {"Name": student, "Date": date_str, "Time": time_str, "Age": "-", "Gender": "-", "Emotion": "-", "Status": "Absent"}
            df = pd.concat([df, pd.DataFrame([absent_entry])], ignore_index=True)
    df.to_excel(file_name, index=False)

current_time = datetime.datetime.now().time()
start_time = datetime.time(9, 30)
end_time = datetime.time(10, 0)

if TEST_MODE or (start_time <= current_time <= end_time):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False)
        face_detector.setInput(blob)
        detections = face_detector.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence < 0.6:
                continue

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype("int")
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w - 1, x2), min(h - 1, y2)
            face_img = frame[y1:y2, x1:x2]

            if face_img.size == 0:
                continue

            face_blob = cv2.dnn.blobFromImage(face_img, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            face_model.setInput(face_blob)
            embedding = face_model.forward().flatten().reshape(1, -1)
            pred = clf.predict(embedding)[0]
            name = le.inverse_transform([pred])[0]

            face_blob2 = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), [104, 117, 123], swapRB=False)
            gender_net.setInput(face_blob2)
            gender = gender_list[gender_net.forward().argmax()]
            age_net.setInput(face_blob2)
            age = age_list[age_net.forward().argmax()]

            gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
            resized_face = cv2.resize(gray_face, (64, 64))
            resized_face = resized_face.astype("float32") / 255.0
            resized_face = np.expand_dims(resized_face, axis=0)
            resized_face = np.expand_dims(resized_face, axis=-1)
            emotion_preds = emotion_model.predict(resized_face, verbose=0)[0]
            emotion = emotion_list[np.argmax(emotion_preds)]

            mark_attendance(name, age, gender, emotion)

            label = f"{name}, {age}, {gender}, {emotion}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2.imshow("Attendance System", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if not TEST_MODE:
        mark_absent_for_missing_students()
else:
    print("Attendance system is only active between 9:30 AM and 10:00 AM.")

Attendance system is only active between 9:30 AM and 10:00 AM.
